In [1]:
import pandas as pd 
import plotly.graph_objects as go
import plotly.express as px
import sys, os
fn_data = "../data/renewables_with_load.parquet"
fn_cap = "../data/renewables_capacity.parquet"
from data import get_generation, normalize_generation, get_profiles
from graphs import plot_profile, plot_daily_generation

2024-02-12 16:11:29.880 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-02-12 16:11:29.881 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-02-12 16:11:29.882 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-02-12 16:11:29.882 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-02-12 16:11:29.883 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-02-12 16:11:29.884 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [2]:
df_gen, df_annual = get_generation(fn_data, fn_cap, country="DE", year=2023)
df_norm = normalize_generation(df_gen, shares={"Wind": 0.4, "Solar": 0.4, "Baseload": 0.2}, total_demand=0)
profiles = get_profiles(df_norm)

2024-02-12 16:11:30.801 
  command:

    streamlit run /Users/jan/git/baseload/venv/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-02-12 16:11:30.802 No runtime found, using MemoryCacheStorageManager
2024-02-12 16:11:30.857 No runtime found, using MemoryCacheStorageManager
2024-02-12 16:11:30.860 No runtime found, using MemoryCacheStorageManager


,Wind,Solar
Capacity,84.377078,208.578191


In [18]:
df_annual["Fullload Hours"]

Solar            875.858628
WindOffshore    2893.486719
WindOnshore     2062.290686
Wind            2165.102337
Demand                  NaN
Name: Fullload Hours, dtype: float64

In [3]:
def get_storage_stats(df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Get curtailment and energy overshoot given the frame of generation

    Args:
        df: frame with hourly generation and demand data

    Returns:
        hourly dataframe with storage statistics; aggregated statistics
    """
    col_gen = [c for c in df.columns if c != "Demand"]
    df_ = df.assign(
        TotalSupply=lambda df: df[col_gen].sum(1),
        ExcessSupply=lambda df: (df["TotalSupply"] - df["Demand"]).map(
            lambda x: max(x, 0)
        ),
        ExcessDemand=lambda df: (df["Demand"] - df["TotalSupply"]).map(
            lambda x: max(x, 0)
        ),
    )
    total = df_.sum()
    stats = {
        var: {
            "Total": total[var],
            "TotalPercentOfDemand": total[var] / total["Demand"] * 100,
            "Max": df_[var].max(),
            "Hours": len(df_[df_[var] > 9]),
        }
        for var in ["ExcessSupply", "ExcessDemand"]
    }
    return df_, pd.DataFrame.from_dict(stats).round(1)

df, stats = get_storage_stats(df_norm)
stats
df_s = df.iloc[:24,:][["ExcessSupply", "ExcessDemand"]].copy()


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8760 entries, 2023-01-01 00:00:00 to 2023-12-31 23:00:00
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Wind          8760 non-null   float64
 1   Solar         8760 non-null   float64
 2   Baseload      8760 non-null   float64
 3   Demand        8760 non-null   float64
 4   TotalSupply   8760 non-null   float64
 5   ExcessSupply  8760 non-null   float64
 6   ExcessDemand  8760 non-null   float64
dtypes: float64(7)
memory usage: 547.5 KB


In [5]:
df_norm.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8760 entries, 2023-01-01 00:00:00 to 2023-12-31 23:00:00
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Wind      8760 non-null   float64
 1   Solar     8760 non-null   float64
 2   Baseload  8760 non-null   float64
 3   Demand    8760 non-null   float64
dtypes: float64(4)
memory usage: 342.2 KB


In [6]:
import numpy as np

In [7]:
# objective: simply minimize the sum
c = np.ones((1, len(df_s)))

# build the constraint matrix
df_s
c

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.]])